First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import cv2
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [ ]:
model.load_state_dict(torch.load('vietdeptrai.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'road_following_model_trt_5.pth')

Load the optimized model by executing the cell below

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt_4.pth'))

<All keys matched successfully>

Create the racecar class

In [3]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [4]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)













Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [6]:
import numpy as np
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from utils import preprocess

car.steering = -0.8
car.steering = 0.8
car.throttle = 0.0
time.sleep(1)

STEERING_GAIN = 1.1
STEERING_BIAS = 0.015
car.throttle = 0
while True:
    # Đọc ảnh từ camera
    image = camera.read()
    processed_image = preprocess(image).half()
    output = model_trt(processed_image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS
#     # Xóa output cũ để cập nhật kết quả mới
    clear_output(wait=True)
    
#     # In thông tin góc lái
    print("Model output (x):", x)
    print(f"Steering: {car.steering:.4f}")



Model output (x): -0.24365234375
Steering: -0.2530


KeyboardInterrupt: 

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 1.5
STEERING_BIAS = 0.1
FIXED_THROTTLE = 0.16  # throttle cố định

car.throttle = FIXED_THROTTLE  # chỉ set 1 lần trước khi chạy

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()

    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS



In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 0.8
STEERING_BIAS = 0.0

BASE_THROTTLE = 0.19  # Tốc độ chạy thẳng
BOOST_WHEN_TURN = 0.018        # Tăng thêm ga khi đánh lái mạnh

THRESHOLD_ANGLE = 0.29      # Nếu góc lái lớn hơn giá trị này → tăng ga

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()

    x = float(output[0])
    steering = x * STEERING_GAIN + STEERING_BIAS
    car.steering = steering

    # Nếu đang đánh lái mạnh → tăng thêm ga để chống tụt áp
    if abs(steering) > THRESHOLD_ANGLE:
        throttle = BASE_THROTTLE + BOOST_WHEN_TURN
    else:
        throttle = BASE_THROTTLE

    # Giới hạn để không vượt mức tối đa
    throttle = min(throttle, 1.0)
    car.throttle = throttle


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from utils import preprocess

kp = 1 
kd = 1
last_x = 0

car.throttle = 0.2
car.throttle = 0.19
car.throttle = 0

while True:
    start_time = time.time()

    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])

    steering = x * kp + (x - last_x) * kd
    steering = max(min(steering, 1.0), -1.0)  # giới hạn [-1, 1]
    car.steering = steering
    last_x = x
   # Xóa output cũ để cập nhật kết quả mới
    clear_output(wait=True)
    
    # In thông tin góc lái
    print("Model output (x):", x)
    print(f"Steering: {car.steering:.4f}")




##### 